In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
!pip install transformers datasets torch

In [13]:
import json
with open('/content/combined_output_v1.json') as f:
    raw_data = json.load(f)

In [35]:
print(raw_data[:5])

[{'instruction': 'Predict the relevant ERPNext Doctype(s) for the question below.', 'input': 'Item stat check plz', 'output': ['Item']}, {'instruction': 'Predict the relevant ERPNext Doctype(s) for the question below.', 'input': 'Rank invoices from most to least for supplier.', 'output': ['Purchase Invoice']}, {'instruction': 'Predict the relevant ERPNext Doctype(s) for the question below.', 'input': 'outline total amnt from all custs due now', 'output': ['Customer']}, {'instruction': 'Predict the relevant ERPNext Doctype(s) for the question below.', 'input': 'Any overdue Stock Entry exist?', 'output': ['Stock Entry']}, {'instruction': 'Predict the relevant ERPNext Doctype(s) for the question below.', 'input': 'Plz gimme those Sales Orders', 'output': ['Sales Order']}]


In [14]:
from datasets import Dataset
raw_data = Dataset.from_list(raw_data)
split_dataset = raw_data.train_test_split(test_size=0.2, seed=42)

In [16]:
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))

Train size: 59576
Validation size: 14895


In [38]:
print(train_dataset.column_names)

['instruction', 'input', 'output']


In [17]:
doctypes = sorted(list(set([','.join(record['output']) if isinstance(record['output'], list) else record['output'] for record in raw_data])))

label2id = {label: idx for idx, label in enumerate(doctypes)}
id2label = {idx: label for label, idx in label2id.items()}
def encode_labels(example):
    example['label'] = label2id[','.join(example['output']) if isinstance(example['output'], list) else example['output']]
    return example

train_dataset = train_dataset.map(encode_labels)
val_dataset = val_dataset.map(encode_labels)
print(doctypes)

Map:   0%|          | 0/59576 [00:00<?, ? examples/s]

Map:   0%|          | 0/14895 [00:00<?, ? examples/s]

['About Us Settings', 'Account', 'Account Closing Balance', 'Accounting Dimension', 'Accounting Dimension Detail', 'Accounting Period', 'Accounts Receivable', 'Accounts Settings', 'Activity Cost', 'Activity Log', 'Activity Type', 'Additional Salary', 'Address', 'Advance Payment Ledger Entry', 'Advance Tax', 'Advance Taxes and Charges', 'Allowed Dimension', 'Allowed To Transact With', 'Applicable On Account', 'Appointment', 'Appointment Booking Settings', 'Appointment Booking Slots', 'Approval', 'Asset', 'Asset Activity', 'Asset Capitalization', 'Asset Capitalization Asset Item', 'Asset Capitalization Service Item', 'Asset Capitalization, Company', 'Asset Capitalization, Payment Entry Reference', 'Asset Capitalization, Process Payment Reconciliation Log', 'Asset Capitalization, Process Payment Reconciliation Log, Company', 'Asset Category', 'Asset Category Account', 'Asset Depreciation Schedule', 'Asset Maintenance', 'Asset Maintenance Log', 'Asset Movement', 'Asset Movement Item', 'Ass

In [18]:
import csv

# Save to CSV format
with open("doctype_mapping.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Doctype", "ID"])  # Header
    for label, idx in label2id.items():
        writer.writerow([label, idx])

print("Saved as doctype_mapping.csv")

Saved as doctype_mapping.csv


In [41]:
train_dataset[4:9]

{'instruction': ['Predict the relevant ERPNext Doctype(s) for the question below.',
  'Predict the relevant ERPNext Doctype(s) for the question below.',
  'Predict the relevant ERPNext Doctype(s) for the question below.',
  'Predict the relevant ERPNext Doctype(s) for the question below.',
  'Predict the relevant ERPNext Doctype(s) for the question below.'],
 'input': ['List stock entries for material receipt.',
  'Can I set a fulfilment deadline for a contract?',
  'Can I view which company was selected in a previous closing voucher?',
  'gimme the count of suppliers',
  'Quantity delivered in this shipment?'],
 'output': [['Stock Entry'],
  ['Contract'],
  ['Period Closing Voucher'],
  ['Supplier'],
  ['Delivery Note']],
 'label': [553, 120, 398, 580, 158]}

In [42]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('hyrinmansoor/text2frappe-s1-roberta')

def preprocess_function(examples):
    return tokenizer(examples['input'], truncation=True, padding="max_length")

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/59533 [00:00<?, ? examples/s]

Map:   0%|          | 0/14884 [00:00<?, ? examples/s]

In [43]:
from huggingface_hub import login

login()

In [44]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_name = "hyrinmansoor/text2frappe-s1-roberta"  # can be swapped anytime
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [45]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification

training_args = TrainingArguments(
    output_dir="./results/ROBERTA-t1",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    report_to="none"
)

# Load the model with the correct number of labels and mappings
model_name = "hyrinmansoor/text2frappe-s1-roberta"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    id2label=id2label,       # Pass the id2label mapping
    label2id=label2id ,
    ignore_mismatched_sizes=True# Pass the label2id mapping
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val
)

# Train the model
trainer.train()

trainer.evaluate()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at hyrinmansoor/text2frappe-s1-roberta and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([829]) in the checkpoint and torch.Size([685]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([829, 768]) in the checkpoint and torch.Size([685, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.610600,0.363756
2,0.159900,0.156835
3,0.097300,0.111043
4,0.065500,0.100142
5,0.042400,0.085186
6,0.022900,0.068440
7,0.016700,0.079783
8,0.016400,0.084691
9,0.011400,0.069396
10,0.005400,0.075918


{'eval_loss': 0.07096367329359055,
 'eval_runtime': 96.5148,
 'eval_samples_per_second': 154.215,
 'eval_steps_per_second': 9.646,
 'epoch': 15.0}

In [47]:
model.save_pretrained("/content/drive/MyDrive/Changai/S1/Model_trained_results/1")
tokenizer.save_pretrained("/content/drive/MyDrive/Changai/S1/Model_trained_results/1")
# model.push_to_hub("hyrinmansoor/text2frappe-s1-roberta")
# tokenizer.push_to_hub("hyrinmansoor/text2frappe-s1-roberta")

('/content/drive/MyDrive/Changai/S1/Model_trained_results/1/tokenizer_config.json',
 '/content/drive/MyDrive/Changai/S1/Model_trained_results/1/special_tokens_map.json',
 '/content/drive/MyDrive/Changai/S1/Model_trained_results/1/vocab.json',
 '/content/drive/MyDrive/Changai/S1/Model_trained_results/1/merges.txt',
 '/content/drive/MyDrive/Changai/S1/Model_trained_results/1/added_tokens.json',
 '/content/drive/MyDrive/Changai/S1/Model_trained_results/1/tokenizer.json')

In [39]:
from huggingface_hub import login

login()

In [ ]:
from huggingface_hub import create_repo

create_repo("text2frappe-s1", private=True)

In [51]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="hyrinmansoor/text2frappe-s1-roberta",
    folder_path="/content/drive/MyDrive/Changai/S1/Model_trained_results/1",
    path_in_repo="",  # root of the model repo
    repo_type="model"
)


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/hyrinmansoor/text2frappe-s1-roberta/commit/1e0f781abc591ccab4f85fee83446b125ff5a8fe', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1e0f781abc591ccab4f85fee83446b125ff5a8fe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hyrinmansoor/text2frappe-s1-roberta', endpoint='https://huggingface.co', repo_type='model', repo_id='hyrinmansoor/text2frappe-s1-roberta'), pr_revision=None, pr_num=None)

In [3]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
# model_path = "/content/drive/MyDrive/Changai/S1/Model"

# model = RobertaForSequenceClassification.from_pretrained(model_path, local_files_only=True)
# tokenizer = RobertaTokenizerFast.from_pretrained(model_path, local_files_only=True)

In [48]:
print(id2label)
with open("id2label.json","w",encoding="utf-8") as f:
  json.dump(id2label,f, indent=4, ensure_ascii=False)

{0: 'About Us Settings', 1: 'Account', 2: 'Account Closing Balance', 3: 'Accounting Dimension', 4: 'Accounting Dimension Detail', 5: 'Accounting Period', 6: 'Accounts Receivable', 7: 'Accounts Settings', 8: 'Activity Cost', 9: 'Activity Log', 10: 'Activity Type', 11: 'Additional Salary', 12: 'Address', 13: 'Advance Payment Ledger Entry', 14: 'Advance Tax', 15: 'Advance Taxes and Charges', 16: 'Allowed Dimension', 17: 'Allowed To Transact With', 18: 'Applicable On Account', 19: 'Appointment', 20: 'Appointment Booking Settings', 21: 'Appointment Booking Slots', 22: 'Approval', 23: 'Asset', 24: 'Asset Activity', 25: 'Asset Capitalization', 26: 'Asset Capitalization Asset Item', 27: 'Asset Capitalization Service Item', 28: 'Asset Capitalization, Company', 29: 'Asset Capitalization, Payment Entry Reference', 30: 'Asset Capitalization, Process Payment Reconciliation Log', 31: 'Asset Capitalization, Process Payment Reconciliation Log, Company', 32: 'Asset Category', 33: 'Asset Category Accoun

In [49]:
import torch
import pandas as pd


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/content/drive/MyDrive/Changai/S1/Model_trained_results/1"


model = RobertaForSequenceClassification.from_pretrained(
    model_path,
    id2label=id2label,
    label2id=label2id,
    local_files_only=True,
    ignore_mismatched_sizes=True  # important!
).to(device)


tokenizer = RobertaTokenizerFast.from_pretrained(model_path, local_files_only=True)
with open("/content/openAI_SD_raw.json","r",encoding="utf-8") as f:
  test_data=json.load(f)

# test_data=[
#   { "input": "new staff joined this yr?", "output": ["Employee"] },
#   { "input": "inactive workers list rn", "output": ["Employee"] },
#   { "input": "who on leave today?", "output": ["Employee"] },

#   { "input": "clients added last 2 wks?", "output": ["Customer"] },
#   { "input": "any cust no orders yet?", "output": ["Customer"] },
#   { "input": "overdue bills clients?", "output": ["Customer"] },

#   { "input": "open so pending deliver?", "output": ["Sales Order"] },
#   { "input": "sales orders this wk?", "output": ["Sales Order"] },
#   { "input": "big so amt rn?", "output": ["Sales Order"] },

#   { "input": "unpaid invoices show", "output": ["Sales Invoice"] },
#   { "input": "avg invoice val mnth?", "output": ["Sales Invoice"] },
#   { "input": "last 5 invoices any cust", "output": ["Sales Invoice"] },

#   { "input": "open po list?", "output": ["Purchase Order"] },
#   { "input": "po made this quarter", "output": ["Purchase Order"] },
#   { "input": "cancelled purchase orders?", "output": ["Purchase Order"] },

#   { "input": "unpaid purchase inv now?", "output": ["Purchase Invoice"] },
#   { "input": "avg pi cost mnth?", "output": ["Purchase Invoice"] },
#   { "input": "top 5 purchase invoices", "output": ["Purchase Invoice"] },

#   { "input": "suppliers dubai list", "output": ["Supplier"] },
#   { "input": "inactive supplier rn", "output": ["Supplier"] },
#   { "input": "supplier max inv", "output": ["Supplier"] },

#   { "input": "projects still running", "output": ["Project"] },
#   { "input": "proj started last mnth", "output": ["Project"] },
#   { "input": "any delayed projects?", "output": ["Project"] },

#   { "input": "low stock items?", "output": ["Item"] },
#   { "input": "products no sale mnth", "output": ["Item"] },
#   { "input": "top 10 selling goods", "output": ["Item"] },

#   { "input": "open tasks today", "output": ["Task"] },
#   { "input": "tasks done last wk", "output": ["Task"] },
#   { "input": "pending tasks hr dept", "output": ["Task"] },

#   { "input": "stock entries today?", "output": ["Stock Entry"] },
#   { "input": "recent stock entry item x", "output": ["Stock Entry"] },
#   { "input": "cancelled stock entries", "output": ["Stock Entry"] },
#    { "input": "new employees last mnth?", "output": ["Employee"] },
#   { "input": "staff still active?", "output": ["Employee"] },
#   { "input": "emp on leave rn", "output": ["Employee"] },

#   { "input": "show me new cust this week", "output": ["Customer"] },
#   { "input": "any clients with 0 invoices", "output": ["Customer"] },
#   { "input": "cust overdue pay list", "output": ["Customer"] },

#   { "input": "sales orders yet to ship", "output": ["Sales Order"] },
#   { "input": "so count today?", "output": ["Sales Order"] },
#   { "input": "biggest sales order amt", "output": ["Sales Order"] },

#   { "input": "inv not paid still", "output": ["Sales Invoice"] },
#   { "input": "avg sales inv val quarter", "output": ["Sales Invoice"] },
#   { "input": "show 10 latest invoices", "output": ["Sales Invoice"] },

#   { "input": "open pos show", "output": ["Purchase Order"] },
#   { "input": "pos created this mnth?", "output": ["Purchase Order"] },
#   { "input": "cancelled pos list", "output": ["Purchase Order"] },

#   { "input": "purchase inv unpaid rn", "output": ["Purchase Invoice"] },
#   { "input": "pi avg amount this quarter", "output": ["Purchase Invoice"] },
#   { "input": "purchase invoices top value", "output": ["Purchase Invoice"] },

#   { "input": "suppliers inactive list", "output": ["Supplier"] },
#   { "input": "supplier with most pos", "output": ["Supplier"] },
#    { "input": "new staff onboard this mnth?", "output": ["Employee"] },
#   { "input": "who is on leave rn", "output": ["Employee"] },
#   { "input": "inactive emp list pls", "output": ["Employee"] },

#   { "input": "new clients joined this wk?", "output": ["Customer"] },
#   { "input": "cust not ordered anything yet", "output": ["Customer"] },
#   { "input": "overdue bills for customers rn", "output": ["Customer"] },

#   { "input": "open so still not shipped", "output": ["Sales Order"] },
#   { "input": "so count today?", "output": ["Sales Order"] },
#   { "input": "highest value sales order rn", "output": ["Sales Order"] },

#   { "input": "show unpaid invoices rn", "output": ["Sales Invoice"] },
#   { "input": "avg invoice amt this quarter", "output": ["Sales Invoice"] },
#   { "input": "10 latest invoices pls", "output": ["Sales Invoice"] },

#   { "input": "open po list rn", "output": ["Purchase Order"] },
#   { "input": "pos created this mnth", "output": ["Purchase Order"] },
#   { "input": "cancelled purchase orders show", "output": ["Purchase Order"] },

#   { "input": "unpaid purchase invs rn", "output": ["Purchase Invoice"] },
#   { "input": "avg purchase invoice val mnth", "output": ["Purchase Invoice"] },
#   { "input": "top 5 purchase invoices val", "output": ["Purchase Invoice"] },

#   { "input": "suppliers in dubai list", "output": ["Supplier"] },
#   { "input": "inactive suppliers rn", "output": ["Supplier"] },
#   { "input": "supplier with max invoices", "output": ["Supplier"] },

#   { "input": "projects still running now", "output": ["Project"] },
#   { "input": "proj started last mnth?", "output": ["Project"] },
#   { "input": "any delayed projects rn", "output": ["Project"] },

#   { "input": "low stock items show", "output": ["Item"] },
#   { "input": "items not sold this mnth", "output": ["Item"] },
#   { "input": "top 10 selling items", "output": ["Item"] },

#   { "input": "open tasks today rn", "output": ["Task"] },
#   { "input": "tasks completed last wk", "output": ["Task"] },
#   { "input": "pending tasks hr dept", "output": ["Task"] },

#   { "input": "stock entries today pls", "output": ["Stock Entry"] },
#   { "input": "recent stock entry item y", "output": ["Stock Entry"] },
#   { "input": "cancelled stock entries list", "output": ["Stock Entry"] }

# ]

results = []
for record in test_data:
    inputs = tokenizer(
        record["input"],
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax(dim=-1).item()

    predicted_doctype = id2label[predicted_class_id]
    results.append({
        "Question": record["input"],
        "Real Answer": record["output"][0],
        "Model Prediction": predicted_doctype,
        "Correct?": "✅" if predicted_doctype == record["output"][0] else "❌"
    })

# Output
df_results = pd.DataFrame(results)
print(df_results)

                      Question Real Answer Model Prediction Correct?
0  new clients joined this wk?    Customer         Customer        ✅


In [34]:
import torch, pandas as pd
with open("/content/openAI_SD_raw.json","r",encoding="utf-8") as f:
  test_data=json.load(f)
correct = 0
total = 0
rows = []
results=[]
for rec in test_data:
    gold = rec["output"][0]
    enc = tokenizer(
        rec["input"],
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(device)

    with torch.no_grad():
        logits = model(**enc).logits
        predicted_class_id = logits.argmax(dim=-1).item()

    predicted_doctype = id2label[predicted_class_id]

    correct += (predicted_doctype == gold)
    total += 1
    results.append({
        "Question": rec["input"],
        "Real Answer": rec["output"][0],
        "Model Prediction": predicted_doctype,
        "Correct?": "✅" if predicted_doctype == rec["output"][0] else "❌"
    })

acc = correct / total if total else 0.0
print("Accuracy:", round(acc, 4))
df_results = pd.DataFrame(results)
df_results.to_csv("/content/results.csv",index=False)


Accuracy: 0.722
